# Script to read TIGGE nc files

In [3]:
import xarray as xr
import os.path
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [4]:
# Heathrow, Bristol, Birmingham, Leeds, Manchester and Glasgow
locations_temp = pd.DataFrame(data={'name': ['Heathrow','Bristol', 'Birmingham', 'Leeds', 'Manchester', 'Glasgow'],
                   'longitude': [-0.449,-2.71285,-1.90493,-1.5979,-2.24264,-4.53122],
                   'latitude': [51.479,51.385,52.4806,53.9282,53.4827,55.9068]})

In [3]:
xr.open_dataset('2022/tigge-GB_20220103.nc')

<xarray.Dataset>
Dimensions:    (longitude: 21, latitude: 19, number: 50, time: 61)
Coordinates:
  * longitude  (longitude) float32 -7.0 -6.5 -6.0 -5.5 -5.0 ... 1.5 2.0 2.5 3.0
  * latitude   (latitude) float32 59.0 58.5 58.0 57.5 ... 51.5 51.0 50.5 50.0
  * number     (number) int32 1 2 3 4 5 6 7 8 9 ... 42 43 44 45 46 47 48 49 50
  * time       (time) datetime64[ns] 2022-01-03 ... 2022-01-18
Data variables:
    u10        (time, number, latitude, longitude) float32 ...
    v10        (time, number, latitude, longitude) float32 ...
    t2m        (time, number, latitude, longitude) float32 ...
    d2m        (time, number, latitude, longitude) float32 ...
    sm         (time, number, latitude, longitude) float32 ...
    sd         (time, number, latitude, longitude) float32 ...
    sf         (time, number, latitude, longitude) float32 ...
    tp         (time, number, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-08-02 12:19:40 GMT by grib_to_netcdf-2.26.0: grib_to_n...

In [4]:
i=1
locations_temp.iloc[i]['longitude']

-2.71285

In [5]:
for year in range(2022,2023):
    #files = os.listdir(f'{year}/')
    #files = [i for i in files if '.nc' in i]

    list_df_issues = []
    files = ['hres-UKNS_20170101.nc']
    for file in files:
        print(file)
        issue_time = pd.to_datetime("hres-UKNS_20170101.nc",utc=True).to_datetime64()
        # print(issue_time)
        
       #ds = xr.open_dataset(f'{year}/{file}')
        ds = xr.open_dataset("PycharmProjects/NGT/hres-UKNS_20170101.nc")
        
        # No need to interpolate to greater time resoultion here... (save memoy and do it only when required)
        # times_out = np.arange(ds.coords['time'].values[0],
        #                       ds.coords['time'].values[0] + np.timedelta64(144, 'h'),
        #                       dtype='datetime64[h]')

        # Only take lead-times up to 144
        ds = ds[["t2m","u10","v10"]].sel(time=slice(ds.coords['time'][0],ds.coords['time'][0] + np.timedelta64(144, 'h')))

        # Interpolate to weather stations
        list_df_locations = []
        for i in range(len(locations_temp)):
            ds_temp = ds.interp(longitude=locations_temp.iloc[i]['longitude'],latitude=locations_temp.iloc[i]['latitude'])
            df_temp = ds_temp.to_dataframe()
            df_temp = df_temp.drop(['longitude','latitude'],axis=1)
            df_temp['location'] = locations_temp.iloc[i]['name']
            list_df_locations.append(df_temp)
        
        df_single_issue = pd.concat(list_df_locations)
        df_single_issue['issue_time'] = issue_time
        
        list_df_issues.append(df_single_issue)

    df_year = pd.concat(list_df_issues)
    df_year.reset_index().to_feather(f'tigge_v0_{year}.feather')

hres-UKNS_20170101.nc


Custom TB Handler failed, unregistering


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "pandas\_libs\tslib.pyx", line 605, in pandas._libs.tslib.array_to_datetime
  File "pandas\_libs\tslibs\parsing.pyx", line 318, in pandas._libs.tslibs.parsing.parse_datetime_string
  File "C:\Users\lalr.tneigroup\AppData\Local\miniconda3\envs\script-encw\Lib\site-packages\dateutil\parser\_parser.py", line 1368, in parse
    return DEFAULTPARSER.parse(timestr, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lalr.tneigroup\AppData\Local\miniconda3\envs\script-encw\Lib\site-packages\dateutil\parser\_parser.py", line 643, in parse
    raise ParserError("Unknown string format: %s", timestr)
dateutil.parser._parser.ParserError: Unknown string format: hres-UKNS_20170101.nc

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "pandas\_libs\tslib.pyx", line 616, in pandas._libs.tslib.array_to_datetime
TypeError: invalid string coercion to datetime for "hres-UKNS_2017